In [36]:
# import dependencies
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials

In [37]:
# authenticate and connect to google sheets
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("api_keys.json", scope)
client = gspread.authorize(creds)
# spreadsheet = client.open("Waypoints A GeoBiography")

In [38]:
# get data from google sheet
sheet = client.open_by_key("12L4EkdRqaQ_e42fGHWaTmgCeqQrNgjTfoeAEc5AB6tw").worksheet("Overview")
overview_data = sheet.get_all_records()

In [39]:
# convert lists stored as strings into actual lists
def parse_list(value):
    if isinstance(value, str) and value.strip():
        return [item.strip() for item in value.split(",")]
    return []

In [40]:
# default values for missing fields
defaults = {
    "estimated": False,
    "photos": [],
    "description": "",
    "importance": "minor"
}

In [41]:
# process data
for entry in overview_data:
    # parse and clean up photos list
    raw_photos = entry.get("photos", "")
    entry["photos"] = [photo.strip('[]"') for photo in parse_list(raw_photos)]
    
    # set default values for missing keys
    for key, default in defaults.items():
        entry.setdefault(key, default)

In [42]:
# save as JSON
with open("./data/overview.json", "w") as file:
    json.dump(overview_data, file, indent=2)

print("Travel data successfully saved!")

Travel data successfully saved!
